# Erosion System Design for HexMagic

## Overview
This document outlines a comprehensive erosion simulation system that integrates with HexMagic's existing watershed/hydrology system and uses Terraform to track erosion as SeismicEvents over geological time.

---

## Core Philosophy

**Key Principle**: Erosion is the inverse of terrain building—material moves from high elevations to low elevations through natural processes (water, ice, wind). By tracking erosion as SeismicEvents, we can:
1. Animate terrain evolution over time
2. Track sediment deposition (negative erosion)
3. Analyze landscape changes
4. Debug erosion parameters

In [ ]:
#| default_exp erosion

In [ ]:
#| export
#standard
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx


In [ ]:
#| export
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexRegion, HexWrapper, HexPosition, PrimitiveDemo, hexBackground
from HexMagic.styles import StyleCSS

In [ ]:
#| export

from HexMagic.hydrology import DrainageBasins, Watershed
from HexMagic.climate import Climate, TerrainFactory
from HexMagic.terrain import Terrain
from HexMagic.terrainpatterns import TerrainPatterns


In [ ]:
#| export
from HexMagic.terraform import Terraform, SeismicEvent, ClimateRenderConfig,  MapRenderConfig

In [ ]:
#| export
@patch
def visualize_erosion(self: Terraform, layer_name="erosion_viz", event_name=None):
    """Visualize erosion with red (removal) and green (deposition)."""
    
    # Get the adjustment from the most recent erosion event
    if event_name is None:
        erosion_events = [e for e in self.events if e.kind == "erosion"]
        if not erosion_events:
            print("No erosion events found")
            return self
        adjustment = erosion_events[-1].adjustment
    else:
        event = next((e for e in self.events if e.name == event_name), None)
        if event is None:
            print(f"Event {event_name} not found")
            return self
        adjustment = event.adjustment
    # 5 categories: heavy erosion, light erosion, minimal, light deposition, heavy deposition
    thresholds = [-5, -1, 1, 5]  # Splits into 5 bins
    
    colors = [
        "#DC143C",  # Red - heavy erosion (< -5m)
        "#DC143C",  # Red - light erosion (-5 to -1m)
        "#808080",  # Gray - minimal change (-1 to 1m)
        "#32CD32",  # Green - light deposition (1 to 5m)
        "#32CD32",  # Green - heavy deposition (> 5m)
    ]
    
    # Bin adjustment values
    adj_indices = np.digitize(adjustment, thresholds)
    adj_indices = np.clip(adj_indices, 0, len(colors) - 1)
    
    # Create patterns with varying ball density
    patternGen = TerrainPatterns(self.terrain)
    patterns = patternGen.ballSpectrum(len(colors), fills=colors, prefix="erosion")
    
    # Generate overlay
    overlay = self.terrain.makeOverlay(adj_indices, patterns)
    self.builder.adjust(layer_name, overlay)
    
    return self


In [ ]:
#| export
class ErrDemo:
    def __init__(self):
        self.help = ""

    @property
    def island(self):
        """Demo: Create island, let it erode over time."""
        
        # Create terrain with climate
        world = TerrainFactory.create_ocean_world(
            bounds=MapRect(MapCord(0,0), MapSize(500, 500)),
            preset='mediterranean',
            radius=15
        )
        terrain = world.terrain
        terrain.hexGrid.adjustRadius(10)
        terrain.colorMap()
        
        # Add volcanic island
        terrain.elevations += terrain.volcano(
            center=414, adjusted=800, num_rings=8,
            variability=0.5, initial_threshold=0.4
        )

        # Add volcanic island
        terrain.elevations += terrain.volcano(
            center=714, adjusted=400, num_rings=8,
            variability=0.5, initial_threshold=0.4
        )
        
        # Compute climate (needed for erosion)
        terrain.climate.configure(terrain, debug=False)
        
        # Run erosion
        return Terraform(terrain)
    

In [ ]:
#| export
@patch
def simpleShow(self:ErrDemo,terra:Terraform,title="Map Example",debug=False):
        # 1. Create blank ocean world with tropical preset
    bounds = MapRect(MapCord(0, 0), MapSize(400, 400))
    world = TerrainFactory.create_ocean_world(
        bounds=bounds,
        preset='tropical',
        radius=15,
        lon_span=5.0,
        num_plates=16,
        debug = debug
    )

    terra = Terraform(world.terrain)

    # maui.add_climate_overlay(layer_name="root")
    rCong = ClimateRenderConfig()
    rCong.show_rivers = True
    rCong.river_colors = ["#1976d2"]  # Default blue
    rCong.show_elevation_borders = True
    rCong.show_coastline  = True
    rCong.mode = "climate_zones" #"hydro"

    mapC = MapRenderConfig(
    title = title,
    background_color = "white" ,
    show_coastline  = True,
    show_hexes = False,
    climate = rCong)

    rCong = ClimateRenderConfig()
   
    terra.config = mapC
    return terra

In [ ]:
#| export
@patch
def example(self:ErrDemo):
    
    terra = self.simpleShow("The Base")
    terra.terrain.colorMap()
    terra.terrain.hexGrid.update()
    return terra.builder.show()

In [ ]:
ErrDemo().example()

In [ ]:
#| export
@dataclass
class ErosionConfig:
    """Configuration for erosion simulation."""
    
    # Time simulation
    years_per_iteration: float = 1000.0  # Geological time per step
    iterations: int = 10
    
    # Fluvial (river) erosion
    fluvial_enabled: bool = True
    fluvial_base_rate: float = 0.01      # m/year base erosion
    fluvial_flow_exponent: float = 0.7   # Flow sensitivity
    fluvial_gradient_exponent: float = 0.5
    fluvial_transport_capacity: float = 2.0  # Sediment carrying capacity
    
    # Glacial erosion
    glacial_enabled: bool = True
    glacial_temp_threshold: float = -2.0  # °C for glacier formation
    glacial_base_rate: float = 0.05       # m/year
    glacial_elevation_threshold: float = 2000.0  # Minimum elevation
    
    # Aeolian (wind) erosion
    aeolian_enabled: bool = True
    aeolian_precip_threshold: float = 250.0  # Below this = desert
    aeolian_base_rate: float = 0.005      # m/year
    aeolian_wind_factor: float = 1.0      # Wind speed multiplier
    
    # Hillslope processes
    hillslope_enabled: bool = True
    hillslope_diffusion: float = 0.01     # Creep/mass wasting
    hillslope_threshold: float = 0.5      # Gradient threshold for landslides
    
    # Coastal erosion
    coastal_enabled: bool = True
    coastal_wave_energy: float = 0.02     # m/year
    coastal_range: int = 2                # Hexes from coastline
    
    # Deposition
    deposition_enabled: bool = True
    deposition_slope_threshold: float = 0.1  # Gentle slopes accumulate
    
    debug: bool = False

In [ ]:
#| export
class ErosionSimulator:
    """Main erosion simulation engine."""
    
    def __init__(self, terraform: Terraform, config: ErosionConfig = None):
        self.terraform = terraform
        self.terrain = terraform.terrain
        self.config = config or ErosionConfig()
        
        # Cache for efficiency
        self._watersheds = None
        self._flow_accumulation = None
        self._gradients = None
    
    def _invalidate_caches(self):
        """Clear cached data between iterations."""
        # Don't clear watersheds - they're stable unless terrain changes drastically
        # self._watersheds = None
        self._flow_accumulation = None
        self._gradients = None
    
    def simulate(self) -> Terraform:
        """Run complete erosion simulation, returning updated Terraform."""
        
        for iteration in range(self.config.iterations):
            if self.config.debug:
                print(f"\n=== Erosion Iteration {iteration + 1}/{self.config.iterations} ===")
            
            # Calculate all erosion types for this iteration
            total_erosion = np.zeros(len(self.terrain.elevations))
            
            if self.config.fluvial_enabled:
                fluvial = self._compute_fluvial_erosion()
                total_erosion += fluvial
                if self.config.debug:
                    print(f"  Fluvial: {np.abs(fluvial).sum():.1f}m total")
            
            if self.config.glacial_enabled:
                glacial = self._compute_glacial_erosion()
                total_erosion += glacial
                if self.config.debug:
                    print(f"  Glacial: {np.abs(glacial).sum():.1f}m total")
            
            if self.config.aeolian_enabled:
                aeolian = self._compute_aeolian_erosion()
                total_erosion += aeolian
                if self.config.debug:
                    print(f"  Aeolian: {np.abs(aeolian).sum():.1f}m total")
            
            if self.config.hillslope_enabled:
                hillslope = self._compute_hillslope_diffusion()
                total_erosion += hillslope
                if self.config.debug:
                    print(f"  Hillslope: {np.abs(hillslope).sum():.1f}m total")
            
            if self.config.coastal_enabled:
                coastal = self._compute_coastal_erosion()
                total_erosion += coastal
                if self.config.debug:
                    print(f"  Coastal: {np.abs(coastal).sum():.1f}m total")
            
            # Handle sediment deposition
            if self.config.deposition_enabled:
                # Fluvial deposition (rivers)
                fluvial_deposition = self._compute_fluvial_deposition(total_erosion)
                total_erosion += fluvial_deposition
                
                # Glacial deposition (ice)
                glacial_deposition = self._compute_glacial_deposition(total_erosion)
                total_erosion += glacial_deposition
                
                if self.config.debug:
                    print(f"  Fluvial deposition: {fluvial_deposition[fluvial_deposition > 0].sum():.1f}m total")
                    print(f"  Glacial deposition: {glacial_deposition[glacial_deposition > 0].sum():.1f}m total")
                    total_dep = fluvial_deposition[fluvial_deposition > 0].sum() + \
                               glacial_deposition[glacial_deposition > 0].sum()
                    total_ero = -total_erosion[total_erosion < 0].sum()
                    print(f"  Mass balance: {total_ero:.1f}m eroded, {total_dep:.1f}m deposited")
            
            # Apply erosion and track as event
            self.terrain.elevations += total_erosion
            
            self.terraform.add_event(
                kind="erosion",
                name=f"erosion_cycle_{iteration}",
                adjustment=total_erosion,
                properties={
                    "years": str(self.config.years_per_iteration),
                    "iteration": str(iteration),
                    "total_removed": str(float(-total_erosion[total_erosion < 0].sum())),
                    "total_deposited": str(float(total_erosion[total_erosion > 0].sum())),
                    "fluvial_deposit": str(float(fluvial_deposition[fluvial_deposition > 0].sum())),
                    "glacial_deposit": str(float(glacial_deposition[glacial_deposition > 0].sum()))
                }
            )
            
            # Invalidate caches for next iteration
            self._invalidate_caches()
        
        return self.terraform

## 1. Fluvial (River) Erosion

### Theory
Rivers erode through:
1. **Hydraulic action**: Water pressure removes material
2. **Abrasion**: Sediment grinds bedrock
3. **Corrosion**: Chemical dissolution

**Key factors**:
- Flow accumulation (discharge)
- Stream gradient (slope)
- Bedrock resistance


In [ ]:
#| export
@patch
def _compute_fluvial_erosion(self:ErosionSimulator) -> np.ndarray:
    """Compute river-driven erosion using watershed data.
    
    Uses Stream Power Law: E = K * A^m * S^n
    Where:
        E = erosion rate
        K = erodibility coefficient
        A = drainage area (flow accumulation)
        S = local slope
        m, n = exponents (typically m=0.5-0.7, n=0.5-1.0)
    """
    
    erosion = np.zeros(len(self.terrain.elevations))
    
    # Get or compute watersheds
    if self._watersheds is None:
        basin = DrainageBasins(self.terrain, debug=False)
        self._watersheds = basin.sheds
    
    # Process each watershed
    for watershed in self._watersheds:
        # Get flow accumulation for this watershed
        flow = watershed.tributary._calculate_flow()
        gradient = watershed.tributary._calculate_gradient()
        
        if not flow:
            continue
        
        # Normalize flow (drainage area proxy)
        max_flow = max(flow.values())
        
        for hex_idx in watershed.tributary.hexes:
            # Skip ocean
            if self.terrain.elevations[hex_idx] <= 0:
                continue
            
            # Get flow and gradient
            A = flow.get(hex_idx, 1)
            S = gradient.get(hex_idx, 0) / 1000.0  # Convert to m/m
            
            if S <= 0:
                continue
            
            # Stream power law
            # Normalize A to [0, 1] range
            A_norm = (A / max_flow) if max_flow > 0 else 0
            
            # E = K * A^m * S^n
            K = self.config.fluvial_base_rate
            m = self.config.fluvial_flow_exponent
            n = self.config.fluvial_gradient_exponent
            
            erosion_rate = K * (A_norm ** m) * (S ** n)
            
            # Scale by time
            erosion_amount = erosion_rate * self.config.years_per_iteration
            
            # Negative = erosion (removal)
            erosion[hex_idx] = -erosion_amount
    
    return erosion

In [ ]:
#| export
@patch
def _get_transport_capacity(self:ErosionSimulator, hex_idx: int, flow: float, gradient: float) -> float:
    """Calculate sediment transport capacity for a hex.
    
    Transport capacity determines how much sediment a river can carry.
    When capacity is exceeded, deposition occurs.
    """
    
    # Shields parameter approach (simplified)
    # Q_s = k * Q^a * S^b
    
    Q = flow  # Discharge proxy
    S = max(gradient / 1000.0, 0.001)  # Slope in m/m
    
    # Transport capacity coefficients
    k = self.config.fluvial_transport_capacity
    a = 1.5  # Discharge exponent
    b = 1.0  # Slope exponent
    
    capacity = k * (Q ** a) * (S ** b)
    
    return capacity

In [ ]:
#| export
@patch
def _compute_fluvial_with_bedrock_resistance(self:ErosionSimulator) -> np.ndarray:
    """Enhanced fluvial erosion with variable rock hardness.
    
    Different rock types erode at different rates.
    """
    
    erosion = self._compute_fluvial_erosion()
    
    # Define bedrock resistance by elevation (proxy for rock type)
    # Lower elevations = sedimentary (soft), higher = igneous (hard)
    for i in range(len(erosion)):
        elev = self.terrain.elevations[i]
        
        if elev > 2000:
            # High mountains: resistant crystalline rocks
            resistance = 0.3
        elif elev > 1000:
            # Mid-elevation: metamorphic rocks
            resistance = 0.6
        elif elev > 500:
            # Hills: consolidated sedimentary
            resistance = 0.8
        else:
            # Lowlands: soft sediments
            resistance = 1.0
        
        erosion[i] *= resistance
    
    return erosion

In [ ]:
#| export
@patch
def _compute_knickpoint_migration(self:ErosionSimulator) -> np.ndarray:
    """Model knickpoint (waterfall) retreat.
    
    Knickpoints are steep steps in river profiles that migrate upstream,
    causing rapid incision.
    """
    
    erosion = np.zeros(len(self.terrain.elevations))
    
    for watershed in self._watersheds:
        gradient = watershed.tributary._calculate_gradient()
        
        # Find knickpoints (sudden gradient increases)
        for hex_idx in watershed.tributary.hexes:
            if self.terrain.elevations[hex_idx] <= 0:
                continue
            
            grad = gradient.get(hex_idx, 0)
            
            # Knickpoint threshold
            if grad > 100:  # >100m drop to next hex
                # Enhanced erosion at knickpoints
                erosion[hex_idx] = -0.5 * self.config.years_per_iteration
                
                # Propagate upstream
                neighbors = self.terrain.ring(hex_idx, 1)
                for n in neighbors:
                    if self.terrain.elevations[n] > self.terrain.elevations[hex_idx]:
                        erosion[n] = -0.2 * self.config.years_per_iteration
    
    return erosion

In [ ]:
!cat ../HexMagic/hydrology.py

I like to refactor these as methods on a drainage basin _compute_fluvial_erosion, _get_transport_capacity, _compute_fluvial_with_bedrock_resistance _compute_knickpoint_migration

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good refactoring instinct — those methods are tightly coupled to watershed/tributary data, so `DrainageBasins` is a natural home.

A few design questions to consider:

1. **Config access**: These methods currently use `self.config` (ErosionConfig). Would you prefer to:
   - Pass `config` as a parameter to each method?
   - Store `config` on `DrainageBasins` (e.g. in `__init__` or a separate `configure_erosion()` call)?

2. **Return scope**: Currently `_compute_fluvial_erosion` loops over all watersheds internally. On `DrainageBasins`, do you want:
   - Methods that operate on the **whole basin collection** (same as now)?
   - Methods on individual `Watershed` objects that you aggregate externally?

What's your preference? Once you decide, I can help you sketch out the signature changes.

I think pass in the config and a list of sheds
we can generate interesting list doing def select_shed(self: DrainageBasins

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Nice — that's a clean design. So the signature pattern would be something like:

```python
@patch
def _compute_fluvial_erosion(self: DrainageBasins, 
                              config: ErosionConfig,
                              sheds: List[Watershed] = None) -> np.ndarray:
```

And you'd call it like:

```python
major_sheds = basins.select_shed(min_flow=10, max_rivers=20)
erosion = basins._compute_fluvial_erosion(config, major_sheds)
```

Default `sheds=None` could mean "use `self.sheds`" for convenience.

Does that match what you're thinking? If so, want me to write out the full refactored `_compute_fluvial_erosion` first?

Yes please refactor

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
@patch
def _compute_fluvial_erosion(self: DrainageBasins, 
                              config: ErosionConfig,
                              sheds: List[Watershed] = None) -> np.ndarray:
    """Compute river-driven erosion using watershed data.
    
    Uses Stream Power Law: E = K * A^m * S^n
    Where:
        E = erosion rate
        K = erodibility coefficient
        A = drainage area (flow accumulation)
        S = local slope
        m, n = exponents (typically m=0.5-0.7, n=0.5-1.0)
    """
    terrain = self.terrain
    erosion = np.zeros(len(terrain.elevations))
    
    if sheds is None:
        sheds = self.sheds
    
    for watershed in sheds:
        flow = watershed.tributary._calculate_flow()
        gradient = watershed.tributary._calculate_gradient()
        
        if not flow:
            continue
        
        max_flow = max(flow.values())
        
        for hex_idx in watershed.tributary.hexes:
            if terrain.elevations[hex_idx] <= 0:
                continue
            
            A = flow.get(hex_idx, 1)
            S = gradient.get(hex_idx, 0) / 1000.0  # Convert to m/m
            
            if S <= 0:
                continue
            
            A_norm = (A / max_flow) if max_flow > 0 else 0
            
            K = config.fluvial_base_rate
            m = config.fluvial_flow_exponent
            n = config.fluvial_gradient_exponent
            
            erosion_rate = K * (A_norm ** m) * (S ** n)
            erosion_amount = erosion_rate * config.years_per_iteration
            erosion[hex_idx] = -erosion_amount
    
    return erosion


@patch
def _get_transport_capacity(self: DrainageBasins,
                            config: ErosionConfig,
                            hex_idx: int, 
                            flow: float, 
                            gradient: float) -> float:
    """Calculate sediment transport capacity for a hex."""
    
    Q = flow
    S = max(gradient / 1000.0, 0.001)
    
    k = config.fluvial_transport_capacity
    a = 1.5
    b = 1.0
    
    return k * (Q ** a) * (S ** b)


@patch
def _compute_fluvial_with_bedrock_resistance(self: DrainageBasins,
                                              config: ErosionConfig,
                                              sheds: List[Watershed] = None) -> np.ndarray:
    """Enhanced fluvial erosion with variable rock hardness."""
    
    terrain = self.terrain
    erosion = self._compute_fluvial_erosion(config, sheds)
    
    for i in range(len(erosion)):
        elev = terrain.elevations[i]
        
        if elev > 2000:
            resistance = 0.3  # High mountains: resistant crystalline
        elif elev > 1000:
            resistance = 0.6  # Mid-elevation: metamorphic
        elif elev > 500:
            resistance = 0.8  # Hills: consolidated sedimentary
        else:
            resistance = 1.0  # Lowlands: soft sediments
        
        erosion[i] *= resistance
    
    return erosion


@patch
def _compute_knickpoint_migration(self: DrainageBasins,
                                   config: ErosionConfig,
                                   sheds: List[Watershed] = None) -> np.ndarray:
    """Model knickpoint (waterfall) retreat."""
    
    terrain = self.terrain
    erosion = np.zeros(len(terrain.elevations))
    
    if sheds is None:
        sheds = self.sheds
    
    for watershed in sheds:
        gradient = watershed.tributary._calculate_gradient()
        
        for hex_idx in watershed.tributary.hexes:
            if terrain.elevations[hex_idx] <= 0:
                continue
            
            grad = gradient.get(hex_idx, 0)
            
            if grad > 100:  # Knickpoint threshold
                erosion[hex_idx] = -0.5 * config.years_per_iteration
                
                neighbors = terrain.ring(hex_idx, 1)
                for n in neighbors:
                    if terrain.elevations[n] > terrain.elevations[hex_idx]:
                        erosion[n] = -0.2 * config.years_per_iteration
    
    return erosion
```

All four now live on `DrainageBasins` with consistent signatures. You can use them like:

```python
basins = DrainageBasins(terrain)
config = ErosionConfig()

# All watersheds
erosion = basins._compute_fluvial_erosion(config)

# Selected major ones
major = basins.get_major(top_n=10)
erosion = basins._compute_fluvial_with_bedrock_resistance(config, major)
```

In [ ]:
#| export
@patch
def _compute_fluvial_erosion(self: DrainageBasins, 
                              config: ErosionConfig,
                              sheds: List[Watershed] = None) -> np.ndarray:
    """Compute river-driven erosion using watershed data.
    
    Uses Stream Power Law: E = K * A^m * S^n
    Where:
        E = erosion rate
        K = erodibility coefficient
        A = drainage area (flow accumulation)
        S = local slope
        m, n = exponents (typically m=0.5-0.7, n=0.5-1.0)
    """
    terrain = self.terrain
    erosion = np.zeros(len(terrain.elevations))
    
    if sheds is None:
        sheds = self.sheds
    
    for watershed in sheds:
        flow = watershed.tributary._calculate_flow()
        gradient = watershed.tributary._calculate_gradient()
        
        if not flow:
            continue
        
        max_flow = max(flow.values())
        
        for hex_idx in watershed.tributary.hexes:
            if terrain.elevations[hex_idx] <= 0:
                continue
            
            A = flow.get(hex_idx, 1)
            S = gradient.get(hex_idx, 0) / 1000.0  # Convert to m/m
            
            if S <= 0:
                continue
            
            A_norm = (A / max_flow) if max_flow > 0 else 0
            
            K = config.fluvial_base_rate
            m = config.fluvial_flow_exponent
            n = config.fluvial_gradient_exponent
            
            erosion_rate = K * (A_norm ** m) * (S ** n)
            erosion_amount = erosion_rate * config.years_per_iteration
            erosion[hex_idx] = -erosion_amount
    
    return erosion


@patch
def _get_transport_capacity(self: DrainageBasins,
                            config: ErosionConfig,
                            hex_idx: int, 
                            flow: float, 
                            gradient: float) -> float:
    """Calculate sediment transport capacity for a hex."""
    
    Q = flow
    S = max(gradient / 1000.0, 0.001)
    
    k = config.fluvial_transport_capacity
    a = 1.5
    b = 1.0
    
    return k * (Q ** a) * (S ** b)


@patch
def _compute_fluvial_with_bedrock_resistance(self: DrainageBasins,
                                              config: ErosionConfig,
                                              sheds: List[Watershed] = None) -> np.ndarray:
    """Enhanced fluvial erosion with variable rock hardness."""
    
    terrain = self.terrain
    erosion = self._compute_fluvial_erosion(config, sheds)
    
    for i in range(len(erosion)):
        elev = terrain.elevations[i]
        
        if elev > 2000:
            resistance = 0.3  # High mountains: resistant crystalline
        elif elev > 1000:
            resistance = 0.6  # Mid-elevation: metamorphic
        elif elev > 500:
            resistance = 0.8  # Hills: consolidated sedimentary
        else:
            resistance = 1.0  # Lowlands: soft sediments
        
        erosion[i] *= resistance
    
    return erosion


@patch
def _compute_knickpoint_migration(self: DrainageBasins,
                                   config: ErosionConfig,
                                   sheds: List[Watershed] = None) -> np.ndarray:
    """Model knickpoint (waterfall) retreat."""
    
    terrain = self.terrain
    erosion = np.zeros(len(terrain.elevations))
    
    if sheds is None:
        sheds = self.sheds
    
    for watershed in sheds:
        gradient = watershed.tributary._calculate_gradient()
        
        for hex_idx in watershed.tributary.hexes:
            if terrain.elevations[hex_idx] <= 0:
                continue
            
            grad = gradient.get(hex_idx, 0)
            
            if grad > 100:  # Knickpoint threshold
                erosion[hex_idx] = -0.5 * config.years_per_iteration
                
                neighbors = terrain.ring(hex_idx, 1)
                for n in neighbors:
                    if terrain.elevations[n] > terrain.elevations[hex_idx]:
                        erosion[n] = -0.2 * config.years_per_iteration
    
    return erosion


In [ ]:
#| export
@patch
def exampleRiver(self:ErrDemo,cycles = 100):
    terra  = self.simpleShow("The Base")
    sim = ErosionSimulator(terra)

    terra.terrain.climate.configure(terra.terrain)
    
    total_erosion = np.zeros(len(terra.terrain.elevations))

    org = terra.terrain.elevations.copy()
    
    for i in range(cycles):
       
        fluvial = sim._compute_fluvial_erosion()
        
        total_erosion +=  fluvial
        terra.terrain.elevations += fluvial

    terra.terrain.elevations = org
    print(f"Erosion range: {total_erosion.min():.3f} to {total_erosion.max():.3f}")
    print(f"Non-zero erosion hexes: {(total_erosion != 0).sum()}")
    # 2. Compute the histogram values and bin edges
    counts, bin_edges = np.histogram(total_erosion, bins=10) # 'bins=10' specifies 10 equal-width bins

    # 3. Print the results (optional)
    print("Histogram counts:", counts)
    print("Bin edges:", bin_edges)

    terra.terrain.elevations += total_erosion
     
    

    #self.simpleShow(terra,f"The rivers after {cycles} cycles: {np.abs(total_erosion).sum():.1f}m total")
    terra.visualize_erosion()

    return terra.terrain.builder.show()


In [ ]:
ErrDemo().exampleRiver(4)


### deposits

In [ ]:
#| export
@patch
def _compute_fluvial_deposition(self:ErosionSimulator, erosion: np.ndarray) -> np.ndarray:
    """Compute comprehensive fluvial deposition.
    
    Creates multiple depositional landforms:
    - Alluvial fans at mountain fronts
    - Floodplains with natural levees
    - Point bars in meander bends
    - Deltas at river mouths
    
    Args:
        erosion: Array of erosion amounts (negative = removal)
    
    Returns:
        Array of deposition amounts (positive = addition)
    """
    
    deposition = np.zeros(len(self.terrain.elevations))
    
    # Calculate total eroded sediment
    total_sediment = -erosion[erosion < 0].sum()
    
    if total_sediment <= 0:
        return deposition
    
    # Get watersheds for routing
    if self._watersheds is None:
        basin = DrainageBasins(self.terrain, debug=False)
        self._watersheds = basin.sheds
    
    # Process each watershed
    for watershed in self._watersheds:
        # Calculate sediment load for this watershed
        watershed_sediment = 0
        for hex_idx in watershed.region.hexes:
            if erosion[hex_idx] < 0:
                watershed_sediment += -erosion[hex_idx]
        
        if watershed_sediment <= 0:
            continue
        
        river = watershed.tributary
        flow = river._calculate_flow()
        gradient = river._calculate_gradient()
        
        remaining_sediment = watershed_sediment
        
        # Sort hexes from source to outlet
        sorted_hexes = sorted(river.hexes, 
                            key=lambda h: flow.get(h, 0), 
                            reverse=True)
        
        prev_gradient = None
        in_floodplain = False
        
        for i, hex_idx in enumerate(sorted_hexes):
            if remaining_sediment <= 0:
                break
            
            current_gradient = gradient.get(hex_idx, 0)
            current_flow = flow.get(hex_idx, 1)
            
            # === 1. ALLUVIAL FANS ===
            # Where steep mountain streams meet flat plains
            if (prev_gradient is not None and 
                prev_gradient > 50 and 
                current_gradient < 10 and
                self.terrain.elevations[hex_idx] > 0):
                
                fan_deposit = remaining_sediment * 0.3
                
                # Deposit at apex (fan head)
                deposition[hex_idx] += fan_deposit * 0.5
                
                # Spread radially (fan spreads out)
                neighbors = self.terrain.ring(hex_idx, 1)
                fan_count = 0
                for n in neighbors:
                    if (n >= 0 and 
                        self.terrain.elevations[n] > 0 and
                        self.terrain.elevations[n] <= self.terrain.elevations[hex_idx]):
                        fan_count += 1
                
                if fan_count > 0:
                    for n in neighbors:
                        if (n >= 0 and 
                            self.terrain.elevations[n] > 0 and
                            self.terrain.elevations[n] <= self.terrain.elevations[hex_idx]):
                            deposition[n] += (fan_deposit * 0.5) / fan_count
                
                remaining_sediment -= fan_deposit
                in_floodplain = True
                
                if self.config.debug:
                    print(f"    Alluvial fan at hex {hex_idx}: {fan_deposit:.2f}m")
            
            # === 2. FLOODPLAINS ===
            # Low-gradient areas with lateral deposition
            if (current_gradient < 5 and 
                self.terrain.elevations[hex_idx] > 0 and
                current_flow > 10):  # Significant flow needed
                
                if not in_floodplain:
                    in_floodplain = True
                
                # Deposit in channel and adjacent areas
                floodplain_deposit = remaining_sediment * 0.1
                
                # Channel deposit (60% in main channel)
                deposition[hex_idx] += floodplain_deposit * 0.6
                
                # Lateral deposit (40% on floodplain)
                # Natural levees form on slightly elevated areas adjacent to channel
                neighbors = self.terrain.ring(hex_idx, 1)
                floodplain_hexes = []
                
                for n in neighbors:
                    if n >= 0 and n not in river.hexes:
                        elev_diff = self.terrain.elevations[n] - self.terrain.elevations[hex_idx]
                        # Only adjacent land hexes at similar elevation
                        if 0 <= elev_diff <= 10 and self.terrain.elevations[n] > 0:
                            floodplain_hexes.append(n)
                
                if floodplain_hexes:
                    levee_deposit = floodplain_deposit * 0.4 / len(floodplain_hexes)
                    for n in floodplain_hexes:
                        # Higher deposition closer to channel (levees)
                        deposition[n] += levee_deposit
                
                remaining_sediment -= floodplain_deposit
            
            # === 3. POINT BARS ===
            # Inside meander bends (where flow slows)
            if (i > 0 and 
                i < len(sorted_hexes) - 1 and
                current_gradient < 3 and
                self.terrain.elevations[hex_idx] > 0):
                
                # Find hexes adjacent to river but not in it
                neighbors = self.terrain.ring(hex_idx, 1)
                
                for n in neighbors:
                    if (n >= 0 and 
                        n not in river.hexes and 
                        self.terrain.elevations[n] > 0):
                        
                        # Check if this could be inside a bend
                        # (simplified: any adjacent hex not in river)
                        bar_deposit = remaining_sediment * 0.05
                        deposition[n] += bar_deposit
                        remaining_sediment -= bar_deposit
                        break  # Only one point bar per location
            
            # === 4. DELTAS ===
            # At river mouth where it meets ocean/lake
            if self.terrain.elevations[hex_idx] <= 0:
                # Ocean/lake - form delta
                delta_deposit = remaining_sediment * 0.7
                
                # Find last land hex (delta apex)
                land_hexes = [h for h in river.hexes if self.terrain.elevations[h] > 0]
                
                if land_hexes:
                    # Get hex with highest flow (closest to mouth)
                    apex = max(land_hexes, key=lambda h: flow.get(h, 0))
                    
                    # Distributary pattern - spread from apex into ocean
                    ocean_neighbors = self.terrain.ring(apex, 1)
                    delta_front = [n for n in ocean_neighbors 
                                  if n >= 0 and self.terrain.elevations[n] <= 0]
                    
                    if delta_front:
                        # Deposit at delta front (builds into ocean)
                        for delta_hex in delta_front:
                            deposition[delta_hex] += delta_deposit * 0.7 / len(delta_front)
                        
                        # Also deposit at apex (delta plain)
                        deposition[apex] += delta_deposit * 0.3
                        
                        if self.config.debug:
                            print(f"    Delta at hex {apex}: {delta_deposit:.2f}m")
                
                remaining_sediment -= delta_deposit
                break
            
            # === 5. CHANNEL BED DEPOSITION ===
            # General deposition based on transport capacity
            if self.terrain.elevations[hex_idx] > 0:
                transport_capacity = self._get_transport_capacity(hex_idx, current_flow, current_gradient)
                
                if transport_capacity < 0.1:
                    # Low capacity - deposit
                    capacity_deficit = (0.1 - transport_capacity) / 0.1
                    channel_deposit = remaining_sediment * capacity_deficit * 0.05
                    deposition[hex_idx] += channel_deposit
                    remaining_sediment -= channel_deposit
            
            prev_gradient = current_gradient
        
        # Any remaining sediment goes to deep ocean (lost to system)
        if remaining_sediment > 0 and self.config.debug:
            print(f"    Sediment to deep ocean: {remaining_sediment:.2f}m")
    
    return deposition

I need _compute_fluvial_deposition on the drainage basins refactores and a simple function on terrain called erodeRivers() that would do all this hydrology

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
@patch
def _compute_fluvial_deposition(self: DrainageBasins,
                                 config: ErosionConfig,
                                 erosion: np.ndarray,
                                 sheds: List[Watershed] = None) -> np.ndarray:
    """Compute comprehensive fluvial deposition.
    
    Creates multiple depositional landforms:
    - Alluvial fans at mountain fronts
    - Floodplains with natural levees
    - Point bars in meander bends
    - Deltas at river mouths
    """
    
    terrain = self.terrain
    deposition = np.zeros(len(terrain.elevations))
    
    total_sediment = -erosion[erosion < 0].sum()
    if total_sediment <= 0:
        return deposition
    
    if sheds is None:
        sheds = self.sheds
    
    for watershed in sheds:
        # Calculate sediment load for this watershed
        watershed_sediment = sum(-erosion[h] for h in watershed.region.hexes if erosion[h] < 0)
        
        if watershed_sediment <= 0:
            continue
        
        river = watershed.tributary
        flow = river._calculate_flow()
        gradient = river._calculate_gradient()
        
        remaining_sediment = watershed_sediment
        
        sorted_hexes = sorted(river.hexes, key=lambda h: flow.get(h, 0), reverse=True)
        
        prev_gradient = None
        in_floodplain = False
        
        for i, hex_idx in enumerate(sorted_hexes):
            if remaining_sediment <= 0:
                break
            
            current_gradient = gradient.get(hex_idx, 0)
            current_flow = flow.get(hex_idx, 1)
            
            # === ALLUVIAL FANS ===
            if (prev_gradient is not None and 
                prev_gradient > 50 and current_gradient < 10 and
                terrain.elevations[hex_idx] > 0):
                
                fan_deposit = remaining_sediment * 0.3
                deposition[hex_idx] += fan_deposit * 0.5
                
                neighbors = terrain.ring(hex_idx, 1)
                fan_neighbors = [n for n in neighbors 
                                if n >= 0 and terrain.elevations[n] > 0 
                                and terrain.elevations[n] <= terrain.elevations[hex_idx]]
                
                if fan_neighbors:
                    per_neighbor = (fan_deposit * 0.5) / len(fan_neighbors)
                    for n in fan_neighbors:
                        deposition[n] += per_neighbor
                
                remaining_sediment -= fan_deposit
                in_floodplain = True
            
            # === FLOODPLAINS ===
            if (current_gradient < 5 and terrain.elevations[hex_idx] > 0 and current_flow > 10):
                in_floodplain = True
                floodplain_deposit = remaining_sediment * 0.1
                deposition[hex_idx] += floodplain_deposit * 0.6
                
                neighbors = terrain.ring(hex_idx, 1)
                floodplain_hexes = [n for n in neighbors 
                                   if n >= 0 and n not in river.hexes
                                   and 0 <= terrain.elevations[n] - terrain.elevations[hex_idx] <= 10
                                   and terrain.elevations[n] > 0]
                
                if floodplain_hexes:
                    levee_deposit = floodplain_deposit * 0.4 / len(floodplain_hexes)
                    for n in floodplain_hexes:
                        deposition[n] += levee_deposit
                
                remaining_sediment -= floodplain_deposit
            
            # === POINT BARS ===
            if (0 < i < len(sorted_hexes) - 1 and 
                current_gradient < 3 and terrain.elevations[hex_idx] > 0):
                
                neighbors = terrain.ring(hex_idx, 1)
                for n in neighbors:
                    if n >= 0 and n not in river.hexes and terrain.elevations[n] > 0:
                        bar_deposit = remaining_sediment * 0.05
                        deposition[n] += bar_deposit
                        remaining_sediment -= bar_deposit
                        break
            
            # === DELTAS ===
            if terrain.elevations[hex_idx] <= 0:
                delta_deposit = remaining_sediment * 0.7
                land_hexes = [h for h in river.hexes if terrain.elevations[h] > 0]
                
                if land_hexes:
                    apex = max(land_hexes, key=lambda h: flow.get(h, 0))
                    ocean_neighbors = terrain.ring(apex, 1)
                    delta_front = [n for n in ocean_neighbors 
                                  if n >= 0 and terrain.elevations[n] <= 0]
                    
                    if delta_front:
                        for delta_hex in delta_front:
                            deposition[delta_hex] += delta_deposit * 0.7 / len(delta_front)
                        deposition[apex] += delta_deposit * 0.3
                
                remaining_sediment -= delta_deposit
                break
            
            # === CHANNEL BED DEPOSITION ===
            if terrain.elevations[hex_idx] > 0:
                transport_capacity = self._get_transport_capacity(config, hex_idx, current_flow, current_gradient)
                
                if transport_capacity < 0.1:
                    capacity_deficit = (0.1 - transport_capacity) / 0.1
                    channel_deposit = remaining_sediment * capacity_deficit * 0.05
                    deposition[hex_idx] += channel_deposit
                    remaining_sediment -= channel_deposit
            
            prev_gradient = current_gradient
    
    return deposition
```

And the simple orchestrator on `Terrain`:

```python
@patch
def erodeRivers(self: Terrain, 
                config: ErosionConfig = None,
                iterations: int = 10,
                use_bedrock_resistance: bool = True,
                debug: bool = False) -> np.ndarray:
    """Simple river erosion over multiple iterations.
    
    Args:
        config: ErosionConfig (uses defaults if None)
        iterations: Number of erosion cycles
        use_bedrock_resistance: Apply variable rock hardness
        debug: Print progress info
    
    Returns:
        Total erosion/deposition array (can be used with Terraform)
    """
    
    if config is None:
        config = ErosionConfig()
    config.debug = debug
    
    # Ensure climate is configured
    if 'precipitation' not in self.fields:
        self.climate.configure(self, debug=debug)
    
    # Compute watersheds once
    basins = DrainageBasins(self, debug=debug)
    
    total_change = np.zeros(len(self.elevations))
    
    for i in range(iterations):
        if debug:
            print(f"\n=== Erosion cycle {i+1}/{iterations} ===")
        
        # Compute erosion
        if use_bedrock_resistance:
            erosion = basins._compute_fluvial_with_bedrock_resistance(config)
        else:
            erosion = basins._compute_fluvial_erosion(config)
        
        # Compute deposition
        deposition = basins._compute_fluvial_deposition(config, erosion)
        
        # Combine
        cycle_change = erosion + deposition
        total_change += cycle_change
        
        # Apply to terrain
        self.elevations += cycle_change
        
        if debug:
            eroded = -erosion[erosion < 0].sum()
            deposited = deposition[deposition > 0].sum()
            print(f"  Eroded: {eroded:.1f}m, Deposited: {deposited:.1f}m")
    
    return total_change
```

Usage is now simple:

```python
# Basic usage
terrain.erodeRivers()

# With options
change = terrain.erodeRivers(
    iterations=50,
    use_bedrock_resistance=True,
    debug=True
)

# Or with custom config
config = ErosionConfig(
    fluvial_base_rate=0.02,
    years_per_iteration=5000
)
terrain.erodeRivers(config=config, iterations=100)
```

In [ ]:
#| export
@patch
def _compute_fluvial_deposition(self: DrainageBasins,
                                 config: ErosionConfig,
                                 erosion: np.ndarray,
                                 sheds: List[Watershed] = None) -> np.ndarray:
    """Compute comprehensive fluvial deposition.
    
    Creates multiple depositional landforms:
    - Alluvial fans at mountain fronts
    - Floodplains with natural levees
    - Point bars in meander bends
    - Deltas at river mouths
    """
    
    terrain = self.terrain
    deposition = np.zeros(len(terrain.elevations))
    
    total_sediment = -erosion[erosion < 0].sum()
    if total_sediment <= 0:
        return deposition
    
    if sheds is None:
        sheds = self.sheds
    
    for watershed in sheds:
        # Calculate sediment load for this watershed
        watershed_sediment = sum(-erosion[h] for h in watershed.region.hexes if erosion[h] < 0)
        
        if watershed_sediment <= 0:
            continue
        
        river = watershed.tributary
        flow = river._calculate_flow()
        gradient = river._calculate_gradient()
        
        remaining_sediment = watershed_sediment
        
        sorted_hexes = sorted(river.hexes, key=lambda h: flow.get(h, 0), reverse=True)
        
        prev_gradient = None
        in_floodplain = False
        
        for i, hex_idx in enumerate(sorted_hexes):
            if remaining_sediment <= 0:
                break
            
            current_gradient = gradient.get(hex_idx, 0)
            current_flow = flow.get(hex_idx, 1)
            
            # === ALLUVIAL FANS ===
            if (prev_gradient is not None and 
                prev_gradient > 50 and current_gradient < 10 and
                terrain.elevations[hex_idx] > 0):
                
                fan_deposit = remaining_sediment * 0.3
                deposition[hex_idx] += fan_deposit * 0.5
                
                neighbors = terrain.ring(hex_idx, 1)
                fan_neighbors = [n for n in neighbors 
                                if n >= 0 and terrain.elevations[n] > 0 
                                and terrain.elevations[n] <= terrain.elevations[hex_idx]]
                
                if fan_neighbors:
                    per_neighbor = (fan_deposit * 0.5) / len(fan_neighbors)
                    for n in fan_neighbors:
                        deposition[n] += per_neighbor
                
                remaining_sediment -= fan_deposit
                in_floodplain = True
            
            # === FLOODPLAINS ===
            if (current_gradient < 5 and terrain.elevations[hex_idx] > 0 and current_flow > 10):
                in_floodplain = True
                floodplain_deposit = remaining_sediment * 0.1
                deposition[hex_idx] += floodplain_deposit * 0.6
                
                neighbors = terrain.ring(hex_idx, 1)
                floodplain_hexes = [n for n in neighbors 
                                   if n >= 0 and n not in river.hexes
                                   and 0 <= terrain.elevations[n] - terrain.elevations[hex_idx] <= 10
                                   and terrain.elevations[n] > 0]
                
                if floodplain_hexes:
                    levee_deposit = floodplain_deposit * 0.4 / len(floodplain_hexes)
                    for n in floodplain_hexes:
                        deposition[n] += levee_deposit
                
                remaining_sediment -= floodplain_deposit
            
            # === POINT BARS ===
            if (0 < i < len(sorted_hexes) - 1 and 
                current_gradient < 3 and terrain.elevations[hex_idx] > 0):
                
                neighbors = terrain.ring(hex_idx, 1)
                for n in neighbors:
                    if n >= 0 and n not in river.hexes and terrain.elevations[n] > 0:
                        bar_deposit = remaining_sediment * 0.05
                        deposition[n] += bar_deposit
                        remaining_sediment -= bar_deposit
                        break
            
            # === DELTAS ===
            if terrain.elevations[hex_idx] <= 0:
                delta_deposit = remaining_sediment * 0.7
                land_hexes = [h for h in river.hexes if terrain.elevations[h] > 0]
                
                if land_hexes:
                    apex = max(land_hexes, key=lambda h: flow.get(h, 0))
                    ocean_neighbors = terrain.ring(apex, 1)
                    delta_front = [n for n in ocean_neighbors 
                                  if n >= 0 and terrain.elevations[n] <= 0]
                    
                    if delta_front:
                        for delta_hex in delta_front:
                            deposition[delta_hex] += delta_deposit * 0.7 / len(delta_front)
                        deposition[apex] += delta_deposit * 0.3
                
                remaining_sediment -= delta_deposit
                break
            
            # === CHANNEL BED DEPOSITION ===
            if terrain.elevations[hex_idx] > 0:
                transport_capacity = self._get_transport_capacity(config, hex_idx, current_flow, current_gradient)
                
                if transport_capacity < 0.1:
                    capacity_deficit = (0.1 - transport_capacity) / 0.1
                    channel_deposit = remaining_sediment * capacity_deficit * 0.05
                    deposition[hex_idx] += channel_deposit
                    remaining_sediment -= channel_deposit
            
            prev_gradient = current_gradient
    
    return deposition


In [ ]:
#| export
@patch
def erodeRivers(self: Terrain, 
                config: ErosionConfig = None,
                iterations: int = 10,
                use_bedrock_resistance: bool = True,
                debug: bool = False) -> np.ndarray:
    """Simple river erosion over multiple iterations.
    
    Args:
        config: ErosionConfig (uses defaults if None)
        iterations: Number of erosion cycles
        use_bedrock_resistance: Apply variable rock hardness
        debug: Print progress info
    
    Returns:
        Total erosion/deposition array (can be used with Terraform)
    """
    
    if config is None:
        config = ErosionConfig()
    config.debug = debug
    
    # Ensure climate is configured
    if 'precipitation' not in self.fields:
        self.climate.configure(self, debug=debug)
    
    # Compute watersheds once
    basins = DrainageBasins(self, debug=debug)
    
    total_change = np.zeros(len(self.elevations))
    
    for i in range(iterations):
        if debug:
            print(f"\n=== Erosion cycle {i+1}/{iterations} ===")
        
        # Compute erosion
        if use_bedrock_resistance:
            erosion = basins._compute_fluvial_with_bedrock_resistance(config)
        else:
            erosion = basins._compute_fluvial_erosion(config)
        
        # Compute deposition
        deposition = basins._compute_fluvial_deposition(config, erosion)
        
        # Combine
        cycle_change = erosion + deposition
        total_change += cycle_change
        
        # Apply to terrain
        self.elevations += cycle_change
        
        if debug:
            eroded = -erosion[erosion < 0].sum()
            deposited = deposition[deposition > 0].sum()
            print(f"  Eroded: {eroded:.1f}m, Deposited: {deposited:.1f}m")
    
    return total_change

In [ ]:
@patch
def example(self:ErrDemo):
    
    terra = self.simpleShow("The Base")
    terrain = terra.terrain

    mountains = terrain.find_peaks(9,0,exclusion_radius=7)
    for i , epicenter in enumerate(mountains):
            
        adjustment = terrain.volcano(center=epicenter, adjusted=200, num_rings=7)
        terrain.elevations += adjustment
            
    hist, bin_edges = np.histogram(terrain.elevations, bins=5)

    print("Histogram:", hist)
    print("Bin edges:", bin_edges)
    
    terrain.elevations, terra.terrain.erodeRivers(iterations=500)
    hist, bin_edges = np.histogram(terrain.elevations, bins=5)

    print("Histogram:", hist)
    print("Bin edges:", bin_edges)
    terrain.colorMap()
    terrain.hexGrid.update()
    basin = DrainageBasins(terrain)
    overlay = ""

    mainBasins = basin.get_major(7)

    for basin in mainBasins:
        small_river = basin.simplify(2)
        small_river.tributary.terrain = terrain
        overlay += basin.draw()

        terrain.hexGrid.builder.adjust("rivers",overlay)
    return terrain.hexGrid.builder.show()
    

In [ ]:
ErrDemo().example()

In [ ]:
#| export
@patch
def exampleRiver(self:ErrDemo,cycles = 100):
    terra  = self.simpleShow("The Base")
    sim = ErosionSimulator(terra)

    terra.terrain.climate.configure(terra.terrain)
    
    total_erosion = np.zeros(len(terra.terrain.elevations))

    org = terra.terrain.elevations.copy()

    sim = ErosionSimulator(terra)
    total_erosion = np.zeros(len(terra.terrain.elevations))

    org = terra.terrain.elevations.copy()
    
    for i in range(cycles):
        terra._watersheds = None
        fluvial = sim._compute_fluvial_erosion()
        
        
        total_erosion +=  fluvial
        terra.terrain.elevations += fluvial

    total_erosion += sim._compute_fluvial_deposition(total_erosion)
    #total_erosion += total_erosion

    terra.terrain.elevations = org
    print(f"Erosion range: {total_erosion.min():.3f} to {total_erosion.max():.3f}")
    print(f"Non-zero erosion hexes: {(total_erosion != 0).sum()}")
    # 2. Compute the histogram values and bin edges
    counts, bin_edges = np.histogram(total_erosion, bins=10) # 'bins=10' specifies 10 equal-width bins

    # 3. Print the results (optional)
    print("Histogram counts:", counts)
    print("Bin edges:", bin_edges)

    terra.add_event(
                kind="erosion",
                name=f"river{cycles}",
                properties={},
                adjustment=total_erosion
            )
    

    #self.simpleShow(terra,f"The rivers after {cycles} cycles: {np.abs(total_erosion).sum():.1f}m total")
    terra.visualize_erosion()

    return terra.builder.show()

In [ ]:
ErrDemo().exampleRiver(400)

Is this river errosion strong enough? I can obviously run it longer, but that takes compute time.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠

Looking at your output:
- **Erosion range**: -371m to +36m over 400 cycles
- **Only 28 hexes** show non-zero erosion

That's quite concentrated erosion—a few hexes are getting carved deeply (up to 371m removed), while most of the terrain is untouched. Whether that's "enough" depends on what you're modeling:

**Questions to consider:**
1. What geological timescale does 400 cycles represent? (400 × `years_per_iteration` = total years)
2. Do you want erosion to spread more broadly across the terrain, or is this concentrated pattern intentional (like deep river valleys)?

The histogram shows most values clustered near zero with a few extreme outliers—that's realistic for river incision, but if you want faster visual results, you might consider increasing `fluvial_base_rate` rather than running more cycles.

What effect are you trying to achieve?

## 2. Glacial Erosion

### Theory
Glaciers erode through:
1. **Plucking**: Freeze-thaw removes rock blocks
2. **Abrasion**: Ice with embedded rocks grinds bedrock
3. **Quarrying**: Pressure fluctuations fracture rock

**Characteristics**:
- U-shaped valleys
- Over-deepening
- Hanging valleys (tributaries)
- Cirques at heads

In [ ]:
#| export
@patch
def _compute_glacial_erosion(self:ErosionSimulator) -> np.ndarray:
    """Compute glacier-driven erosion.
    
    Glaciers form in cold, high-elevation areas and flow downhill,
    eroding much more effectively than rivers.
    """
    
    erosion = np.zeros(len(self.terrain.elevations))
    
    # Check if temperature field exists
    if 'temperature' not in self.terrain.fields:
        return erosion
    
    temp = self.terrain.fields['temperature']
    
    # Identify glacier zones
    glacier_mask = (temp < self.config.glacial_temp_threshold) & \
                   (self.terrain.elevations > self.config.glacial_elevation_threshold)
    
    glacier_hexes = np.where(glacier_mask)[0]
    
    if len(glacier_hexes) == 0:
        return erosion
    
    # Compute ice flow direction (downhill)
    for hex_idx in glacier_hexes:
        current_elev = self.terrain.elevations[hex_idx]
        
        # Find steepest descent direction
        neighbors = self.terrain.ring(hex_idx, 1)
        max_gradient = 0
        
        for neighbor in neighbors:
            if neighbor < 0 or neighbor >= len(self.terrain.elevations):
                continue
            
            neighbor_elev = self.terrain.elevations[neighbor]
            gradient = current_elev - neighbor_elev
            
            if gradient > max_gradient:
                max_gradient = gradient
        
        # Glacial erosion rate (scales with slope)
        # Steeper slopes = faster ice flow = more erosion
        gradient_factor = min(max_gradient / 100.0, 2.0)  # Cap at 2x
        
        erosion_rate = self.config.glacial_base_rate * (1.0 + gradient_factor)
        erosion_amount = erosion_rate * self.config.years_per_iteration
        
        erosion[hex_idx] = -erosion_amount
        
        # Glaciers also erode downslope path
        lowest = self.terrain.lowest_neighbor(hex_idx)
        if lowest is not None and self.terrain.elevations[lowest] > 0:
            erosion[lowest] = -erosion_amount * 0.5
    
    return erosion

In [ ]:
#| export
@patch
def _identify_cirques(self:ErosionSimulator) -> list[int]:
    """Identify cirque (bowl) locations at glacier heads.
    
    Cirques form at glacier sources through freeze-thaw and rotational slip.
    """
    
    if 'temperature' not in self.terrain.fields:
        return []
    
    temp = self.terrain.fields['temperature']
    cirques = []
    
    # Find peaks in cold areas
    peaks = self.terrain.find_peaks(k=50, min_height=1500)
    
    for peak in peaks:
        if temp[peak] < self.config.glacial_temp_threshold:
            cirques.append(peak)
    
    return cirques

In [ ]:
#| export
@patch
def _compute_cirque_erosion(self:ErosionSimulator) -> np.ndarray:
    """Enhanced erosion at cirque locations."""
    
    erosion = np.zeros(len(self.terrain.elevations))
    cirques = self._identify_cirques()
    
    for cirque_center in cirques:
        # Erode cirque bowl
        for ring_num in range(1, 4):  # 3-ring radius
            ring_hexes = self.terrain.ring(cirque_center, ring_num)
            
            for hex_idx in ring_hexes:
                if self.terrain.elevations[hex_idx] <= 0:
                    continue
                
                # More erosion closer to center
                intensity = 1.0 - (ring_num / 4.0)
                erosion_rate = 0.1 * intensity
                erosion[hex_idx] = -erosion_rate * self.config.years_per_iteration
    
    return erosion

In [ ]:
#| export
@patch
def _compute_valley_widening(self:ErosionSimulator) -> np.ndarray:
    """Model U-shaped valley formation by glaciers.
    
    Glaciers erode valley sides as well as bottoms, creating
    characteristic U-shaped cross-sections.
    """
    
    erosion = np.zeros(len(self.terrain.elevations))
    
    # For each glacier hex, erode neighbors laterally
    glacier_mask = (self.terrain.fields['temperature'] < self.config.glacial_temp_threshold) & \
                   (self.terrain.elevations > self.config.glacial_elevation_threshold)
    
    glacier_hexes = np.where(glacier_mask)[0]
    
    for hex_idx in glacier_hexes:
        neighbors = self.terrain.ring(hex_idx, 1)
        
        for neighbor in neighbors:
            if neighbor < 0 or self.terrain.elevations[neighbor] <= 0:
                continue
            
            # Lateral erosion (valley widening)
            lateral_rate = self.config.glacial_base_rate * 0.3
            erosion[neighbor] = -lateral_rate * self.config.years_per_iteration
    
    return erosion

### Deposits

In [ ]:
#| export
@patch
def _compute_glacial_deposition(self:ErosionSimulator, erosion: np.ndarray) -> np.ndarray:
    """Compute glacial deposition (moraines, till, outwash).
    
    Glaciers transport eroded material and deposit it when:
    1. Ice melts at terminus (terminal moraine)
    2. Ice melts at sides (lateral moraine)
    3. Ice stagnates (ground moraine/till)
    4. Meltwater loses energy (outwash plain)
    
    Args:
        erosion: Array including glacial erosion (negative values)
    
    Returns:
        Array of glacial deposition (positive values)
    """
    
    deposition = np.zeros(len(self.terrain.elevations))
    
    if 'temperature' not in self.terrain.fields:
        return deposition
    
    temp = self.terrain.fields['temperature']
    
    # Identify glacier zones
    glacier_mask = (temp < self.config.glacial_temp_threshold) & \
                   (self.terrain.elevations > self.config.glacial_elevation_threshold)
    
    glacier_hexes = set(np.where(glacier_mask)[0])
    
    if not glacier_hexes:
        return deposition
    
    # Calculate total sediment eroded by glaciers
    glacial_sediment = 0
    for hex_idx in glacier_hexes:
        if erosion[hex_idx] < 0:
            glacial_sediment += -erosion[hex_idx]
    
    if glacial_sediment <= 0:
        return deposition
    
    if self.config.debug:
        print(f"    Total glacial sediment: {glacial_sediment:.2f}m")
    
    # === 1. TERMINAL MORAINES ===
    # Deposited at glacier terminus (snout/front)
    # Find glacier edges where ice flows downslope
    
    terminal_zones = []
    for hex_idx in glacier_hexes:
        neighbors = self.terrain.ring(hex_idx, 1)
        
        for n in neighbors:
            if n < 0 or n >= len(self.terrain.elevations):
                continue
            
            # Found edge: glacier hex next to non-glacier land
            if n not in glacier_hexes and self.terrain.elevations[n] > 0:
                # Check if downslope (terminus, not lateral edge)
                if self.terrain.elevations[n] < self.terrain.elevations[hex_idx]:
                    terminal_zones.append(n)
    
    # Deposit at terminal positions
    if terminal_zones:
        terminal_deposit = glacial_sediment * 0.4  # 40% forms terminal moraine
        terminal_hexes = list(set(terminal_zones))  # Remove duplicates
        
        for hex_idx in terminal_hexes:
            # Higher deposits where more ice converges
            # Count how many glacier hexes drain to this terminal
            contributing = sum(1 for tz in terminal_zones if tz == hex_idx)
            weight = contributing / len(terminal_zones)
            deposition[hex_idx] += terminal_deposit * weight
        
        if self.config.debug:
            print(f"    Terminal moraine: {terminal_deposit:.2f}m at {len(terminal_hexes)} locations")
    
    # === 2. LATERAL MORAINES ===
    # Deposited along glacier sides (valley walls)
    
    lateral_zones = []
    for hex_idx in glacier_hexes:
        neighbors = self.terrain.ring(hex_idx, 1)
        
        for n in neighbors:
            if n < 0 or n >= len(self.terrain.elevations):
                continue
            
            if n not in glacier_hexes and self.terrain.elevations[n] > 0:
                # Lateral edge: same elevation or higher (valley wall)
                if self.terrain.elevations[n] >= self.terrain.elevations[hex_idx]:
                    lateral_zones.append(n)
    
    if lateral_zones:
        lateral_deposit = glacial_sediment * 0.2  # 20% forms lateral moraines
        lateral_hexes = list(set(lateral_zones))
        
        for hex_idx in lateral_hexes:
            deposition[hex_idx] += lateral_deposit / len(lateral_hexes)
        
        if self.config.debug:
            print(f"    Lateral moraine: {lateral_deposit:.2f}m at {len(lateral_hexes)} locations")
    
    # === 3. GROUND MORAINE (TILL) ===
    # Unsorted debris deposited beneath glacier
    # Spread thinly over entire glacier footprint
    # This is what remains when ice retreats
    
    till_deposit = glacial_sediment * 0.3  # 30% forms till
    
    for hex_idx in glacier_hexes:
        # Thicker till in valleys (lower elevations)
        neighbors = self.terrain.ring(hex_idx, 1)
        if neighbors:
            neighbor_elevs = [self.terrain.elevations[n] 
                            for n in neighbors 
                            if n >= 0 and n < len(self.terrain.elevations)]
            if neighbor_elevs:
                avg_neighbor = np.mean(neighbor_elevs)
                # If lower than neighbors, it's in a valley - more till
                valley_factor = 1.0
                if self.terrain.elevations[hex_idx] < avg_neighbor:
                    valley_factor = 1.5
                
                deposition[hex_idx] += (till_deposit / len(glacier_hexes)) * valley_factor
    
    if self.config.debug:
        print(f"    Ground moraine (till): {till_deposit:.2f}m distributed")
    
    # === 4. OUTWASH PLAINS (GLACIOFLUVIAL) ===
    # Glacial meltwater carries sediment beyond glacier
    # Deposits sorted (stratified) layers as water loses energy
    
    outwash_deposit = glacial_sediment * 0.1  # 10% forms outwash
    
    if terminal_zones:
        terminal_hexes = list(set(terminal_zones))
        
        # For each terminal position, trace meltwater path
        for terminus_hex in terminal_hexes:
            current = terminus_hex
            outwash_remaining = outwash_deposit / len(terminal_hexes)
            
            # Meltwater flows downhill, depositing progressively
            for step in range(5):  # Outwash extends several hexes downstream
                if outwash_remaining <= 0.01:
                    break
                
                if current < 0 or self.terrain.elevations[current] <= 0:
                    break
                
                # Deposit decreasing amounts (coarse first, fine later)
                # Coarse material drops out first
                deposit_fraction = 0.4 - (step * 0.05)  # Decreasing deposition rate
                deposit_here = outwash_remaining * max(deposit_fraction, 0.1)
                deposition[current] += deposit_here
                outwash_remaining -= deposit_here
                
                # Move downhill
                lowest = self.terrain.lowest_neighbor(current)
                if lowest is None:
                    break
                
                current = lowest
        
        if self.config.debug:
            print(f"    Outwash plain: {outwash_deposit:.2f}m distributed")
    
    return deposition

In [ ]:
#| export
@patch
def _compute_glacial_erratics(self:ErosionSimulator) -> list[tuple[int, float]]:
    """Identify locations of erratic boulders.
    
    Erratics are large boulders carried by glaciers and deposited
    far from their source rock. Used for visualization/flavor.
    
    Returns:
        List of (hex_index, boulder_size_meters) tuples
    """
    
    if 'temperature' not in self.terrain.fields:
        return []
    
    temp = self.terrain.fields['temperature']
    
    # Find areas that were glaciated
    glacier_mask = (temp < self.config.glacial_temp_threshold) & \
                   (self.terrain.elevations > self.config.glacial_elevation_threshold)
    
    glacier_hexes = set(np.where(glacier_mask)[0])
    
    erratics = []
    
    # Randomly place erratics in glaciated and terminal moraine areas
    for hex_idx in glacier_hexes:
        if random.random() < 0.05:  # 5% chance per hex
            boulder_size = random.uniform(1.0, 5.0)  # 1-5 meters
            erratics.append((hex_idx, boulder_size))
    
    return erratics


## 3. Aeolian (Wind) Erosion

### Theory
Wind erosion occurs through:
1. **Deflation**: Fine particles lifted and carried away
2. **Abrasion**: Saltating particles sandblast surfaces
3. **Attrition**: Particles break into finer pieces

**Key factors**:
- Low vegetation (low precipitation)
- Loose sediment availability
- Wind speed and direction
- Lack of moisture

In [ ]:
#| export
@patch
def _compute_aeolian_erosion(self:ErosionSimulator) -> np.ndarray:
    """Compute wind-driven erosion in arid regions.
    
    Wind erosion is most effective in deserts where:
    - Precipitation < 250mm/year
    - Vegetation is sparse
    - Loose sediments are available
    """
    
    erosion = np.zeros(len(self.terrain.elevations))
    
    # Check for precipitation data
    if 'precipitation' not in self.terrain.fields:
        return erosion
    
    precip = self.terrain.fields['precipitation']
    
    # Identify arid zones
    arid_mask = (precip < self.config.aeolian_precip_threshold) & \
                (self.terrain.elevations > 0) & \
                (self.terrain.elevations < 1500)  # Wind erosion mainly lowlands
    
    arid_hexes = np.where(arid_mask)[0]
    
    if len(arid_hexes) == 0:
        return erosion
    
    # Get climate for wind data
    if not hasattr(self.terrain, 'climate'):
        wind_speed = 10.0  # Default m/s
    else:
        wind_speed = self.terrain.climate.wind_speed
    
    wind_factor = (wind_speed / 10.0) * self.config.aeolian_wind_factor
    
    for hex_idx in arid_hexes:
        # Erosion inversely proportional to precipitation
        # (less rain = less vegetation = more erosion)
        precip_factor = 1.0 - (precip[hex_idx] / self.config.aeolian_precip_threshold)
        precip_factor = max(0, precip_factor)
        
        # Elevation factor (flatter = more susceptible)
        neighbors = self.terrain.ring(hex_idx, 1)
        elevation_variance = 0
        for n in neighbors:
            if n >= 0 and n < len(self.terrain.elevations):
                elevation_variance += abs(self.terrain.elevations[n] - 
                                         self.terrain.elevations[hex_idx])
        
        elevation_variance /= len(neighbors)
        flatness = 1.0 / (1.0 + elevation_variance / 100.0)
        
        # Combined erosion rate
        erosion_rate = self.config.aeolian_base_rate * wind_factor * \
                       precip_factor * flatness
        
        erosion_amount = erosion_rate * self.config.years_per_iteration
        erosion[hex_idx] = -erosion_amount
    
    return erosion

In [ ]:
#| export
@patch
def _compute_dune_migration(self:ErosionSimulator) -> np.ndarray:
    """Model sand dune formation and movement.
    
    In deserts, erosion creates sand that accumulates into dunes.
    Dunes migrate in the wind direction.
    """
    
    adjustment = np.zeros(len(self.terrain.elevations))
    
    if 'precipitation' not in self.terrain.fields:
        return adjustment
    
    precip = self.terrain.fields['precipitation']
    
    # Very arid areas form dunes
    dune_mask = (precip < 100) & \
                (self.terrain.elevations > 0) & \
                (self.terrain.elevations < 500)
    
    dune_hexes = np.where(dune_mask)[0]
    
    if not hasattr(self.terrain, 'climate'):
        return adjustment
    
    wind_dir = self.terrain.climate.wind_dir  # degrees
    
    for hex_idx in dune_hexes:
        # Erode windward side
        adjustment[hex_idx] = -0.01 * self.config.years_per_iteration
        
        # Deposit on leeward side
        # Find hex in downwind direction
        # (simplified - use wind_dir to pick direction)
        neighbors = self.terrain.ring(hex_idx, 1)
        
        # Pick neighbor closest to downwind direction
        # (this is a simplification - actual implementation would use
        # wind_dir to calculate proper direction)
        if len(neighbors) > 0:
            leeward = neighbors[int(wind_dir / 60) % len(neighbors)]
            adjustment[leeward] = +0.01 * self.config.years_per_iteration
    
    return adjustment

## 4. Hillslope Processes

### Theory
Hillslope erosion includes:
1. **Soil creep**: Slow downslope movement
2. **Mass wasting**: Landslides, slumps
3. **Rain splash**: Particle detachment
4. **Sheet wash**: Overland flow erosion

In [ ]:
#| export
@patch
def _compute_hillslope_diffusion(self:ErosionSimulator) -> np.ndarray:
    """Compute hillslope erosion via diffusion equation.
    
    Models gradual downslope sediment transport (creep, solifluction).
    Uses diffusion: dz/dt = K * ∇²z
    
    Where K is the hillslope diffusion coefficient.
    """
    
    erosion = np.zeros(len(self.terrain.elevations))
    K = self.config.hillslope_diffusion
    
    for i in range(len(self.terrain.elevations)):
        if self.terrain.elevations[i] <= 0:
            continue  # Skip ocean
        
        # Get neighbors
        neighbors = self.terrain.ring(i, 1)
        
        if len(neighbors) == 0:
            continue
        
        # Calculate Laplacian (curvature)
        z_center = self.terrain.elevations[i]
        z_neighbors = []
        
        for n in neighbors:
            if n >= 0 and n < len(self.terrain.elevations):
                z_neighbors.append(self.terrain.elevations[n])
        
        if not z_neighbors:
            continue
        
        # Laplacian approximation
        z_mean = np.mean(z_neighbors)
        laplacian = z_mean - z_center
        
        # Diffusion equation
        erosion_rate = K * laplacian
        erosion[i] = erosion_rate * self.config.years_per_iteration
    
    return erosion

In [ ]:
#| export
@patch
def _compute_mass_wasting(self:ErosionSimulator) -> np.ndarray:
    """Model landslides on steep slopes.
    
    When slope exceeds critical angle, material fails and moves downslope.
    """
    
    erosion = np.zeros(len(self.terrain.elevations))
    
    for i in range(len(self.terrain.elevations)):
        if self.terrain.elevations[i] <= 0:
            continue
        
        # Calculate maximum gradient to neighbors
        neighbors = self.terrain.ring(i, 1)
        max_gradient = 0
        steepest_neighbor = None
        
        for n in neighbors:
            if n < 0 or n >= len(self.terrain.elevations):
                continue
            
            gradient = (self.terrain.elevations[i] - self.terrain.elevations[n]) / \
                      self.terrain.hexGrid.hexes[0].radius
            
            if gradient > max_gradient:
                max_gradient = gradient
                steepest_neighbor = n
        
        # If slope exceeds threshold, trigger landslide
        if max_gradient > self.config.hillslope_threshold and steepest_neighbor is not None:
            # Amount of material that fails
            failure_amount = (max_gradient - self.config.hillslope_threshold) * \
                           self.config.years_per_iteration * 10
            
            # Remove from source
            erosion[i] = -failure_amount
            
            # Deposit at base (with some loss)
            erosion[steepest_neighbor] = failure_amount * 0.8
    
    return erosion

## 5. Coastal Erosion

### Theory
Coasts erode through:
1. **Wave action**: Hydraulic pressure, abrasion
2. **Salt weathering**: Chemical breakdown
3. **Tidal cycles**: Repeated wetting/drying

In [ ]:
#| export
@patch
def _compute_coastal_erosion(self:ErosionSimulator) -> np.ndarray:
    """Compute wave-driven coastal erosion.
    
    Erodes land hexes near the coastline, with intensity
    decreasing inland.
    """
    
    erosion = np.zeros(len(self.terrain.elevations))
    
    # Find coastline (land adjacent to ocean)
    for i in range(len(self.terrain.elevations)):
        if self.terrain.elevations[i] <= 0:
            continue  # Skip ocean
        
        # Check if adjacent to ocean
        neighbors = self.terrain.ring(i, 1)
        adjacent_to_ocean = any(self.terrain.elevations[n] <= 0 
                               for n in neighbors if n >= 0)
        
        if adjacent_to_ocean:
            # Direct coastal hex - maximum erosion
            wave_energy = self.config.coastal_wave_energy
            erosion[i] = -wave_energy * self.config.years_per_iteration
            
            # Also erode hexes inland (spray zone)
            for ring_dist in range(1, self.config.coastal_range + 1):
                inland_hexes = self.terrain.ring(i, ring_dist)
                
                for inland in inland_hexes:
                    if inland < 0 or self.terrain.elevations[inland] <= 0:
                        continue
                    
                    # Erosion decreases with distance
                    intensity = 1.0 - (ring_dist / (self.config.coastal_range + 1))
                    erosion[inland] = -wave_energy * intensity * \
                                     self.config.years_per_iteration
    
    return erosion

In [ ]:
#| export
@patch
def _compute_cliff_retreat(self:ErosionSimulator) -> np.ndarray:
    """Model cliff erosion and notch formation.
    
    Steep coastal cliffs retreat through undermining and collapse.
    """
    
    erosion = np.zeros(len(self.terrain.elevations))
    
    for i in range(len(self.terrain.elevations)):
        if self.terrain.elevations[i] <= 100:  # Need some height for cliff
            continue
        
        neighbors = self.terrain.ring(i, 1)
        
        # Check for steep drop to ocean
        for n in neighbors:
            if n < 0 or n >= len(self.terrain.elevations):
                continue
            
            if self.terrain.elevations[n] <= 0:  # Ocean neighbor
                # Cliff height
                cliff_height = self.terrain.elevations[i]
                
                if cliff_height > 50:
                    # Taller cliffs erode faster (undermining)
                    retreat_rate = 0.05 * (cliff_height / 100.0)
                    erosion[i] = -retreat_rate * self.config.years_per_iteration
                    break
    
    return erosion

## Put it together

In [ ]:
#| export
@patch
def erosion_timelapse(self: ErrDemo, total_years=1000, snapshot_interval=100,debug = False):
    """Show erosion progression with periodic snapshots."""
    terra  = self.simpleShow("The Base")
    
    
    # Configure erosion
    config = ErosionConfig(
        years_per_iteration=100,  # Each iteration = 100 years
        iterations=1,              # Run one at a time
        debug=debug
    )
    org = terra.terrain.clone()
    terra.terrain.climate.configure(terra.terrain)
    simulator = ErosionSimulator(terra, config)
    
    # Store snapshots
    snapshots = []
    num_snapshots = total_years // snapshot_interval
    
    for i in range(num_snapshots):
        # Accumulate erosion for this interval
        snapshot_erosion = np.zeros(len(terra.terrain.elevations))
        
        # Run erosion for this period
        simulator.simulate()
        
        # Get the most recent event
        last_event = terra.events[-1]
        snapshot_erosion += last_event.adjustment
        
        # Store snapshot
        terra.add_event(
            kind="erosion",
            name=f"year_{(i+1)*snapshot_interval}",
            adjustment=snapshot_erosion,
            properties={"years": str((i+1)*snapshot_interval)}
        )
        if debug:
            print(f"Year {(i+1)*snapshot_interval}: {np.abs(snapshot_erosion).sum():.1f}m total change")
    
     
    terrain = terra.terrainFromEvents()
    terrain.colorMap()
    terrain.hexGrid.update()
    return org, terrain


In [ ]:

start, finish = ErrDemo().erosion_timelapse()
start.colorMap()
start.hexGrid.update()
start.hexGrid.builder.show()

In [ ]:
finish.colorMap()
finish.hexGrid.update()
finish.hexGrid.builder.show()

## 6. Sediment Deposition

### Theory
Eroded material must go somewhere:
1. **Fluvial**: Deposited where flow slows (deltas, floodplains, alluvial fans, point bars)
2. **Glacial**: Deposited at glacier terminus (moraines, till, outwash)
3. **Aeolian**: Deposited in dunes, loess
4. **Marine**: Deposited offshore (continental shelf)

**Key Principle**: Sediment mass balance must be maintained:
```
Total erosion = Total deposition + sediment_to_deep_ocean
```

---

## 6.1 Fluvial Deposition

### Landforms Created
1. **Alluvial Fans**: Where steep streams meet flat areas
2. **Floodplains**: Lateral spreading in low-gradient areas
3. **Point Bars**: Inside meander bends
4. **Natural Levees**: Raised banks along channels
5. **Deltas**: At river mouths

### Implementation

## 6.2 Glacial Deposition

### Landforms Created
1. **Terminal Moraines**: Ridges at glacier snout (furthest extent)
2. **Lateral Moraines**: Ridges along glacier sides
3. **Ground Moraine (Till)**: Unsorted debris beneath/within glacier
4. **Outwash Plains**: Sorted sediments from meltwater
5. **Erratics**: Large boulders transported far from source